In [2]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np


In [4]:

races_df = pd.read_csv('races.csv')

In [5]:
races_df = races_df.drop(['fp1_date','fp1_time','fp2_date','fp2_time','fp3_date','fp3_time','quali_date','quali_time','sprint_date','sprint_time'], axis=1)

In [6]:
print(races_df.columns)
races_df.head()
# 


Index(['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time', 'url'], dtype='object')


,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


In [7]:
print(races_df.shape)

(1102, 8)


In [8]:
new_races_df = races_df[races_df["year"] <= 2021]
new_races_df = new_races_df[races_df["year"] >= 2000]
print(new_races_df.shape)
new_races_df.head()

(411, 8)


C:\Users\Akhil Jayadeep\AppData\Local\Temp\ipykernel_23016\321429984.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_races_df = new_races_df[races_df["year"] >= 2000]


,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


In [9]:
weather_df = new_races_df[["raceId", "year", "round","circuitId","name"]]
weather_df.shape

(411, 5)

In [10]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

info = []

for link in new_races_df.url:
    try:
        res = requests.get(link)
        soup = BeautifulSoup(res.content, 'html.parser')
        
        # find the infobox
        infobox = soup.find('table', {'class': 'infobox vevent'})
        
        # find the weather header in the infobox
        weather_header = infobox.find('th', {'class': 'infobox-label', 'scope': 'row', 'style': 'width:20%;'}, text='Weather')
        
        # get the weather information from the corresponding cell
        if weather_header:
            weather_info = weather_header.find_next_sibling('td').get_text(strip=True)
            info.append(weather_info)
            # print(f"Weather: {weather_info}")
        else:
            info.append('not found')
            # print("Weather: not found")
    except:
        info.append('not found')
        # print("Weather: not found")

C:\Users\Akhil Jayadeep\AppData\Local\Temp\ipykernel_23016\1040750503.py:17: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  weather_header = infobox.find('th', {'class': 'infobox-label', 'scope': 'row', 'style': 'width:20%;'}, text='Weather')


In [11]:
len(info)
weather_df['weather'] = info

C:\Users\Akhil Jayadeep\AppData\Local\Temp\ipykernel_23016\2489759733.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_df['weather'] = info


In [12]:
weather_df

,raceId,year,round,circuitId,name,weather
0,1,2009,1,1,Australian Grand Prix,Sunny with temperatures reaching up to 27 °C (...
1,2,2009,2,2,Malaysian Grand Prix,"Dry start, with heavy rain and thunderstorm/mo..."
2,3,2009,3,17,Chinese Grand Prix,Rain
3,4,2009,4,3,Bahrain Grand Prix,Sunny
4,5,2009,5,4,Spanish Grand Prix,"Warm, Sunny"
...,...,...,...,...,...,...
1053,1069,2021,17,69,United States Grand Prix,Sunny
1054,1070,2021,18,32,Mexico City Grand Prix,Sunny
1055,1071,2021,19,18,São Paulo Grand Prix,Sunny
1056,1072,2021,21,77,Saudi Arabian Grand Prix,Clear


In [13]:
counts = weather_df['weather'].value_counts()

# get the count for the "not found" value
not_found_count = counts['not found']
print(not_found_count)

1


In [14]:
weather_dict = {'weather_warm': ['clear', 'warm', 'hot', 'sunny', 'fine', 'mild','not found'],
               'weather_cold': ['cold', 'fresh', 'chilly', 'cool'],
               'weather_dry': ['dry','not found'],
               'weather_wet': ['showers', 'wet', 'rain', 'damp', 'thunderstorms', 'rainy'],
               'weather_cloudy': ['overcast', 'clouds', 'cloudy', 'grey']}

In [15]:
weather_df2 = pd.DataFrame(columns = weather_dict.keys())

In [16]:
for col in weather_df2.columns:
    weather_df2[col] = weather_df['weather'].str.lower().str.contains('|'.join(weather_dict[col]), na=False).astype(int)


In [17]:
weather_df2.head()

,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,1,0,0,0,0
1,0,0,1,1,0
2,0,0,0,1,0
3,1,0,0,0,0
4,1,0,0,0,0


In [18]:
weather_final = pd.concat([weather_df, weather_df2], axis = 1)

In [19]:
weather_final



,raceId,year,round,circuitId,name,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,1,2009,1,1,Australian Grand Prix,Sunny with temperatures reaching up to 27 °C (...,1,0,0,0,0
1,2,2009,2,2,Malaysian Grand Prix,"Dry start, with heavy rain and thunderstorm/mo...",0,0,1,1,0
2,3,2009,3,17,Chinese Grand Prix,Rain,0,0,0,1,0
3,4,2009,4,3,Bahrain Grand Prix,Sunny,1,0,0,0,0
4,5,2009,5,4,Spanish Grand Prix,"Warm, Sunny",1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1053,1069,2021,17,69,United States Grand Prix,Sunny,1,0,0,0,0
1054,1070,2021,18,32,Mexico City Grand Prix,Sunny,1,0,0,0,0
1055,1071,2021,19,18,São Paulo Grand Prix,Sunny,1,0,0,0,0
1056,1072,2021,21,77,Saudi Arabian Grand Prix,Clear,1,0,0,0,0


In [20]:
weather_final.to_csv('weather_final.csv', index=False)